<a href="https://colab.research.google.com/github/noriakihanya/getmusic/blob/main/Getmusic_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.13 numpy tensorboard miditoolkit scipy matplotlib tqdm transformers einops
!git clone https://github.com/microsoft/muzic.git

#!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 .
!pip install note-seq
!pip install pyfluidsynth
!pip install midi2audio
!apt-get update -qq && apt-get install -qq libfluidsynth2 build-essential libasound2-dev libjack-dev fluidsynth

!git clone https://github.com/noriakihanya/getmusic.git
import shutil
shutil.copy("/content/getmusic/track_generation.py","/content/muzic/getmusic/")

import gdown
gdown.download('https://drive.google.com/uc?id=1Ub45GMApdXJAGt0_RRbPWwlhIVIHVcT7', 'checkpoint.pth', quiet=False)
gdown.download('https://drive.google.com/uc?id=1wVauTFZFFCqbiFzRAK2Th_Kn0p5HzOiG', 'MuseScore_General.sf2', quiet=False)

In [2]:
import os
os.makedirs("/content/01_input", exist_ok=True)
os.makedirs("/content/02_output_midi", exist_ok=True)
os.makedirs("/content/03_output_mp3", exist_ok=True)

In [ ]:
#@title ファイルのアップロード
%cd /content/01_input

from google.colab import files
uploaded = files.upload()

In [ ]:
#@title トラックの生成
%cd /content/muzic/getmusic

!python /content/muzic/getmusic/track_generation.py \
--load_path /content/checkpoint.pth \
--file_path /content/input \
--output_midi /content/output_midi
--select_condition_tracks p \
--select_content_tracks bdgs

/content/muzic/getmusic
Global seed set to 0
<class 'getmusic.modeling.models.dfm.DFM'>
<class 'getmusic.modeling.roformer.diffusion_roformer.DiffusionRFM'>
<class 'getmusic.modeling.roformer.roformer_utils.DiffusionRoformerModel'>
Load dictionary: 11879 tokens.
<class 'getmusic.engine.clip_grad_norm.ClipGradNorm'>
Get lr 3e-06 from base lr 3e-06 with none
<class 'torch.optim.adamw.AdamW'>
<class 'getmusic.engine.lr_scheduler.LinearDecayLRWithWarmup'>
{'overall': {'trainable': '86.49M', 'non_trainable': '96.0K', 'total': '86.59M'}, 'rfm': {'trainable': '86.49M', 'non_trainable': '96.0K', 'total': '86.59M'}}
self.device  0
inference_cache: global rank 0: prepare solver done!
Resume from /content/checkpoint.pth
/content/input/よぞらに.mid
skip?n
Select condition tracks ('b' for bass, 'd' for drums, 'g' for guitar, 'l' for lead, 'p' for piano, 's' for strings, 'c' for chords; multiple choices; input any other key to skip):p
Select content tracks ('b' for bass, 'd' for drums, 'g' for guitar, '

In [ ]:
#@title 高度な操作
%cd /content/muzic/getmusic

!python /content/muzic/getmusic/position_generation.py \
--load_path /content/checkpoint.pth \
--file_path /content/muzic/getmusic/example_data/input

#Input positions you want to condition on: 0,0,200;6,0,
#Input positions you want to empty: 1,0,;4,0,;5,0,

In [ ]:
#@title MIDIの表示
import note_seq

SAMPLE_RATE = 16000
SF2_PATH = '/content/MuseScore_General.sf2'

import glob
for i in glob.glob(f'/content/input/よぞらに.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    midi_scr = i

est_ns = note_seq.midi_file_to_note_sequence(midi_scr)

note_seq.play_sequence(est_ns, synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(est_ns)

/content/input/よぞらに.mid


In [ ]:
#@title 伴奏を抽出
import mido
from mido import Message, MidiFile, MidiTrack

for i in glob.glob(f'/content/output_midi/*.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    midi = mido.MidiFile(i)

gomi = MidiFile()
mid = MidiFile()
track = MidiTrack()

i = 0
for t in midi.tracks:
  i += 1
  gomi.tracks.append(t)
  if i >=3:
    mid.tracks.append(t)

mid.save('/content/output_midi/acp_tracks.mid')

/content/output_midi/p2bdgs-よぞらに.mid


In [ ]:
#@title MIDIをオーディオフファイルに変換
from pydub import AudioSegment
from midi2audio import FluidSynth
import glob
import math

for i in glob.glob(f'/content/output_midi/acp_tracks.mid'):
#for i in glob.glob(f'/content/muzic/getmusic/example_data/input/position-*.mid'):
    print(i)
    MTF_FILE = i

fs = FluidSynth(sound_font="/content/MuseScore_General.sf2")
MTF_mp3 = "/content/output_mp3/acp.mp3"
fs.midi_to_audio(MTF_FILE, MTF_mp3)

audio = AudioSegment.from_file(MTF_mp3)
# 音量(%)
volume_percent = 500

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export(MTF_mp3, format='mp3')

/content/output_midi/acp_tracks.mid


<_io.BufferedRandom name='/content/output_mp3/acp.mp3'>